# Inspect GLASS models and generate synthetic images

## Imports

In [ ]:
import sys
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
root = "/Users/phdenzel/gleam"
sys.path.append(root)
import gleam
from gleam.lensobject import LensObject
from gleam.utils.lensing import LensModel
from gleam.reconsrc import ReconSrc, run_model
from gleam.utils.plotting import kappa_map_plot, kappa_profiles_plot
from gleam.utils.plotting import arrival_time_surface_plot
from gleam.utils.plotting import plot_scalebar, plot_labelbox
from gleam.utils.rgb_map import radial_mask
import gleam.utils.colors as gcl
gcl.GLEAMcmaps.register_all()


## Load data files

In [ ]:
#        ACS          ACS(WFC3)        ACS(WFC3)      WFC3(UNK)
#        ACS(WFC3)    SBC              WFC3           ACS(GAIA)
objs = ['B1608+656', 'DESJ0408-5354', 'HE0435-1223', 'PG1115+080',
        'RXJ0911+0551', 'RXJ1131-1231', 'SDSSJ1004+4112', 'WFIJ2033-4723']
objidx = 0
sigf = 80     #   80,  100,   60,  600,  140, 4000,   80,   80
lens = objs[objidx]
print(lens)


In [ ]:
fitsdir = 'data/delay_qsos/'
fitsfile = fitsdir + '{}.fits'.format(lens)
print(fitsfile)

jsondir = 'jsons/'
jsonfile = jsondir + '{}.json'.format(lens)
print(jsonfile)


In [ ]:
with open(jsonfile) as f:
    lo = LensObject.from_json(f)
lo.squarify(0)
print(lo.__v__)


In [ ]:
statedir = 'states/'
statefiles = ['11doubles_dg45.state',
              '11doubles_dg60.state', '11doubles_CMB_dg60.state', '11doubles_SNeIa_dg60.state',
              '7quads_dg45.state', '7quads_CMB_dg45.state', '7quads_SNeIa_dg45.state',
              '7quads_dg60.state', '7quads_CMB_dg60.state', '7quads_SNeIa_dg60.state', 
              'all_dg60.state', 'all_SNeIa_dg60.state']
statefile = statefiles[9]  # 7  8/9 5/6
print(statefile)


In [ ]:
lm = LensModel(statedir+statefile)
lm.obj_idx = objidx
print(lm.__v__)


## Model plots

### Kappa maps

In [ ]:
#%%script false
# for i in range(lm.N_obj):
#     lm.obj_idx = i
i = objidx
kappa_map_plot(lm, obj_index=i, extent=lm.extent, contours=True, levels=7, delta=0.1, label=lm.obj_name)
plot_scalebar(R=lm.maprad, length=max(int(lm.maprad/2), 1))
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.savefig('results/{}_{}_kappa.pdf'.format(lm.obj_name, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
plt.close()
plt.show()


### Kappa profiles

In [ ]:
#%%script false
# for i in range(lm.N_obj):
#     lm.obj_idx = i
i = objidx
kappa_profiles_plot(lm, obj_index=i, ensemble_average=True, refined=True, interpolate=150, levels=10,
                    as_range=True, maprad=lm.maprad, pixrad=lm.pixrad,
                    adjust_limits=True, annotation_color='white', label_axes=True, fontsize=22)
plt.tight_layout()
plt.savefig('results/{}_{}_profiles.pdf'.format(lm.obj_name, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
# plt.close()
plt.show()


In [ ]:
%%script false
fig, axes = plt.subplots(4, 2, sharex=False, sharey=True, figsize=(7, 11))
for i in range(len(objs)):
    lm.obj_idx = i
    print(i)
    yextent = [0.65, 3.00]
    plt.sca(axes[i // 2][i % 2])
    plots, _, _ = kappa_profiles_plot(lm, obj_index=i, ensemble_average=True, refined=True,
                                      interpolate=150, levels=10, as_range=True, maprad=lm.maprad,
                                      pixrad=lm.pixrad, adjust_limits=True, annotation_color='white',
                                      einstein_radius_indicator=True, kappa1_line=True,
                                      label_axes=False, fontsize=22)
    xlim = list(plt.gca().axes.get_xlim())
    plt.contourf(np.ones((4, 4))*plots[0].levels[0], extent=xlim+yextent,
                 cmap='agaveglitch', levels=plots[0].levels, zorder=-99)
    plot_labelbox(objs[i], position='top right', padding=(0.03, 0.04), color='white', fontsize=8)
    plt.ylim(*yextent)
    # plt.rcParams['mathtext.fontset'] = 'stixsans'
    if (i % 2) == 0:
        if (i // 2) == 2:
            axes[i // 2][i % 2].set_ylabel(r'$\mathsf{\kappa}_{<\mathsf{R}}$', fontsize=22)
    if (i // 2) > 2:
        if (i % 2) == 0:
            axes[i // 2][i % 2].set_xlabel(r'R [arcsec]', fontsize=16)
#     else:
#         axes[i // 2][i % 2].set_xticklabels([""]*len(axes[i // 2][i % 2].axes.get_xticklabels()))
    axes[i // 2][i % 2].set_frame_on(True)

axes[3][0].xaxis.set_label_coords(1, -0.175)
axes[2][0].yaxis.set_label_coords(-0.1, 1)
fig.subplots_adjust(wspace=0.025)
# plt.savefig("profiles.pdf", transparent=True, bbox_inches='tight', pad_inches=0)
# plt.close()
plt.show()

### Arrvial time surfaces

In [ ]:
#%%script false
# for i in range(lm.N_obj):
#     lm.obj_idx = i
i = objidx
arrival_time_surface_plot(lm, obj_index=i, draw_images=True,
                          contours=True, levels=60, min_contour_shift=None,
                          sad_contour_shift=None, scalebar=False, label=lm.obj_name, color='black')
# plt.plot(0, 0, color=gcl.blue, marker='+', markersize=16)
plot_scalebar(R=lm.maprad, length=max(int(lm.maprad/2), 1), color='black')
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.savefig('results/{}_{}_arriv.pdf'.format(lm.obj_name, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
# plt.close()
plt.show()


In [ ]:
%%script false
print("Loading files...")

lm_H0 = LensModel(statedir+statefiles[7])
lm_H0.obj_idx = objidx
lm.obj_idx = objidx

print("Calculating arrival time grids...")
arriv = lm_H0.arrival_grid(model_index=-1)    # variable H0
arriv_fixed = lm.arrival_grid(model_index=-1) # CMB/SNeIa


In [ ]:
%%script false
# lm.obj_idx = objidx
# lm_H0.obj_idx = objidx
# arriv = lm_H0.arrival_grid(model_index=-1)
# arriv_fixed = lm.arrival_grid(model_index=-1)
tsqr2days = 365.25/lm.env.nu[0] * lm.env.objects[objidx].dL * (1+lm.env.objects[objidx].z)
print tsqr2days

delta = (arriv - arriv_fixed) * tsqr2days
print(delta.shape)
rmsk = radial_mask(delta, radius=37.5)
edge = np.abs(delta[rmsk].max())
vmin = -edge
vmax = edge
plt.imshow(delta, cmap='twilight', extent=lm.extent, origin='upper', vmin=vmin, vmax=vmax)
# plt.imshow(diffs, cmap='phoenix', extent=lm.extent, origin='upper', vmin=vmin, vmax=vmax)
plot_scalebar(R=lm.maprad, length=max(int(lm.maprad/2), 1), color='white')
plot_labelbox(objs[objidx], position='top right', padding=(0.03, 0.04), color='white', fontsize=14)
cbar = plt.colorbar()
cbar.ax.set_ylabel('[d]', rotation=0)
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.show()

In [ ]:
%%script false
limits = []
for i in range(len(objs)):
    lm.obj_idx = i
    lm_H0.obj_idx = i
    arriv = lm_H0.arrival_grid(model_index=-1)    # variable H0
    arriv_fixed = lm.arrival_grid(model_index=-1) # CMB/SNeIa

    tsqr2days = 365.25/lm.env.nu[0] * lm.env.objects[objidx].dL * (1+lm.env.objects[objidx].z)
    delta = (arriv - arriv_fixed) * tsqr2days
    rmsk = radial_mask(delta, radius=42.5)
    edge = np.abs(delta[rmsk]).max()
    vmin, vmax = -edge, edge
    cmapstr = 'twilight'
    plt.imshow(delta, cmap=cmapstr, extent=lm.extent, origin='upper', vmin=vmin, vmax=vmax)

    minima = lm.minima
    saddles = lm.saddle_points
    maxima = lm.maxima
    cmap = plt.get_cmap(cmapstr)
    min_clr, sad_clr, max_clr = cmap(.8), cmap(.5), cmap(.2)
    if len(minima) > 0:
        plt.plot(minima.T[0], minima.T[1], color=min_clr, marker='o', lw=0)
    if len(saddles) > 0:
        plt.plot(saddles.T[0], saddles.T[1], color=sad_clr, marker='o', lw=0)
    if len(maxima) > 0:
        plt.plot(maxima.T[0], maxima.T[1], color=max_clr, marker='o', lw=0)

    plot_scalebar(R=lm.maprad, length=max(int(lm.maprad/2), 1), color='white')
    plot_labelbox(objs[i], position='top right', padding=(0.03, 0.04), color='white', fontsize=14)
    cbar = plt.colorbar()
    cbar.ax.set_ylabel('[d]', rotation=0)
    plt.axis('off')
    plt.gcf().axes[0].get_xaxis().set_visible(False)
    plt.gcf().axes[0].get_yaxis().set_visible(False)
    plt.tight_layout()
    plt.savefig('results/{}_{}_arrivdelta.pdf'.format(lm.obj_name, lm.filename.replace('.state', '')),
                transparent=True, bbox_inches='tight', pad_inches=0)
    plt.show()
    plt.close()


### Hubble constant

In [ ]:
def H02aHz(H0arr):
    H0arr = np.asarray(H0arr)  # assume in units of km/s/Mpc
    H0 = H0arr.copy()
    H0 /= 3.08567758e1         # aHz=1e-18 sec
    return H0

def aHz2H0(vH0):
    vH0 *= 3.08567758e1
    return vH0

def H02Gyrs(H0arr):
    H0arr = np.asarray(H0arr)    # assume in units of km/s/Mpc
    invH0 = 1./H0arr.copy()      # s Mpc/km
    invH0 *= 3.08567758e3/np.pi  # Gyrs (Mpc=3.08567758e22 m; Gyr=pi*1e16 sec; km=1e3 m)
    return invH0

def H02critdens(H0arr):
    c = 299792458.0            # m s^-1
    G = 6.67430e-11            # kg^-1 m^3 s^-2
    GeV = 1.602176634e-10      # J/GeV
    H0arr = np.asarray(H0arr)  # assume in units of km/s/Mpc
    H02 = H0arr.copy()**2
    H02 /= 3.08567758e19*3.08567758e19  # s^-2
    c2_8piG = 3*c**2/(8*np.pi*G) # kg m^2 m^-3
    rhocrit = H02 * c2_8piG    # J / m^3
    rhocrit /= 1.602176634e-10 # GeV / m^3
    return rhocrit


aHzH0 = H02aHz(lm.H0)
print(aHzH0[0])

invH0 = H02Gyrs(lm.H0)
print(invH0[0])

rhocrit = H02critdens(lm.H0)
print(rhocrit[0])


In [ ]:
#%%script false
q = np.percentile(aHzH0, [16, 50, 84])

fig, ax = plt.subplots()
axc = ax.twiny()

# second unit axis
def convertaxc(ax):
    x1, x2 = ax.get_xlim()
    axc.set_xlim(aHz2H0(x1), aHz2H0(x2))
    axc.figure.canvas.draw()
ax.callbacks.connect("xlim_changed", convertaxc)

n, bins, patches = ax.hist(aHzH0, bins=30, density=True, rwidth=0.901)
cm = plt.cm.get_cmap('phoenix')
ax.axvline(q[1], color=gcl.pink)
ax.axvline(q[0], color=gcl.red)
ax.axvline(q[2], color=gcl.red)

# add gaussian color scheme to pdf
bin_centers = 0.5 * (bins[:-1] + bins[1:])
x = np.exp(-(bin_centers - q[1])**2/np.mean(np.diff(q))**2)
x = x/x.max()
for c, p in zip(x, patches):
    plt.setp(p, 'facecolor', cm(c))
# add result text
plt.rcParams['mathtext.fontset'] = 'stixsans'
Hstr = 'H$_0$ = ${:5.1f}^{{{:+4.1f}}}_{{{:+4.1f}}}$'
q = np.percentile(lm.H0, [16, 50, 84])
plt.text(0.02, 0.85, Hstr.format(q[1], np.diff(q)[1], -np.diff(q)[0]), fontsize=19, color='black', transform=plt.gca().transAxes)

fig.axes[0].get_yaxis().set_visible(False)
ax.set_xlabel('H$_0$ [aHz]')
axc.set_xlabel('H$_0$ [km/s/Mpc]', fontsize=12)
plt.setp(axc.get_xticklabels(), fontsize=12)
plt.tight_layout()
# plt.savefig('results/_H0hist_{}.pdf'.format(statefile.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
# plt.close()
plt.show()


In [ ]:
#%%script false
q = np.percentile(invH0, [16, 50, 84])
n, bins, patches = plt.hist(invH0, bins=30, density=True, rwidth=0.901)
cm = plt.cm.get_cmap('phoenix')
plt.axvline(q[1], color=gcl.pink)
plt.axvline(q[0], color=gcl.red)
plt.axvline(q[2], color=gcl.red)

# add gaussian color scheme to pdf
bin_centers = 0.5 * (bins[:-1] + bins[1:])
x = np.exp(-(bin_centers - q[1])**2/np.mean(np.diff(q))**2)
x = x/x.max()
for c, p in zip(x, patches):
    plt.setp(p, 'facecolor', cm(c))
plt.rcParams['mathtext.fontset'] = 'stixsans'
Hstr = 'H$_0^{{-1}}$ = ${:5.1f}^{{{:+4.1f}}}_{{{:+4.1f}}}$'
plt.text(0.62, 0.85, Hstr.format(q[1], np.diff(q)[1], -np.diff(q)[0]), fontsize=19, color='black', transform=plt.gca().transAxes)

plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.xlabel('H$_0^{-1}$ [Gyr]')
plt.tight_layout()
# plt.savefig('results/_invH0hist_{}.pdf'.format(statefile.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
# plt.close()
plt.show()

In [ ]:
#%%script false
q = np.percentile(rhocrit, [16, 50, 84])
n, bins, patches = plt.hist(rhocrit, bins=30, density=True, rwidth=0.901)
cm = plt.cm.get_cmap('phoenix')
plt.axvline(q[1], color=gcl.pink)
plt.axvline(q[0], color=gcl.red)
plt.axvline(q[2], color=gcl.red)

# add gaussian color scheme to pdf
bin_centers = 0.5 * (bins[:-1] + bins[1:])
x = np.exp(-(bin_centers - q[1])**2/np.mean(np.diff(q))**2)
x = x/x.max()
for c, p in zip(x, patches):
    plt.setp(p, 'facecolor', cm(c))
plt.rcParams['mathtext.fontset'] = 'stixsans'
Hstr = r"$\rho_{{\mathrm{{c}}}}$ = ${:5.1f}^{{{:+4.1f}}}_{{{:+4.1f}}}$"
plt.text(0.02, 0.85, Hstr.format(q[1], np.diff(q)[1], -np.diff(q)[0]), fontsize=19, color='black', transform=plt.gca().transAxes)

plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.xlabel(r"$\rho_{\mathrm{c}}$ [GeV/m$^{{3}}$]")
plt.tight_layout()
# plt.savefig('results/_H02denshist_{}.pdf'.format(statefile.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
# plt.close()
plt.show()


## Source reconstruction

In [ ]:
# lo.data = np.fliplr(lo.data)
reconsrc = ReconSrc(lo, lm, M=60, M_fullres=400, mask_keys=['circle'])
reconsrc.chmdl(-1)
sig2 = sigf * np.abs(reconsrc.lensobject.data)
sig2[sig2 == 0] = sig2[sig2 != 0].min()

print reconsrc.lensobject.data.shape


In [ ]:
reconsrc.calc_psf("psf/tinytim_ACS.fits", normalize=True, window_size=8, verbose=True)
# reconsrc.calc_psf("psf/tinytim_SBC.fits", normalize=True, window_size=8, verbose=True)
# reconsrc.calc_psf("psf/tinytim_WFC3.fits", normalize=True, window_size=8, verbose=True)
# reconsrc.calc_psf("psf/tinytim_WFC3.fits", normalize=True, window_size=4, verbose=True)


In [ ]:
# reconsrc.rotation = 0.95*reconsrc.lensobject.hdr['ORIENTAT']
# reconsrc.rotation = -4.0  # reconsrc.lensobject.hdr['ORIENTAT']
# reconsrc.rotation = -4.5  # reconsrc.lensobject.hdr['ORIENTAT']


In [ ]:
kw = dict(method='minres', use_psf=True, use_mask=True, use_filter=False, sigma2=sig2.copy(), # nonzero_only=True, within_radius=0.8
          cached=True, from_cache=False, save_to_cache=False)
          # cached=True, from_cache=True, save_to_cache=True)


In [ ]:
%%script false
dbeta, xy, r_max, r_fullres = reconsrc.inv_proj_matrix(use_mask=True, return_props=True)

plt.hist2d(xy[:, 0], xy[:, 1])
plt.show()


In [ ]:
%%script false
from __future__ import print_function
qs = []
for i in range(-1, reconsrc.model.N):
    reconsrc.chmdl(i)
    reconsrc.inv_proj_matrix(use_mask=True, r_max=2)
    chi2 = reconsrc.reproj_chi2(reduced=False, **kw)
    qs.append(chi2)
    message = "{:8.4f} @ {:04d}".format(chi2, i)
    print(message, end="\r")


In [ ]:
%%script false
np.savetxt("{}_{}_chi2.txt".format(lm.filename.replace('.state', ''), lens), qs)

In [ ]:
%%script false
#qs = np.loadtxt("results/7quads_CMB_dg60_B1608+656_chi2.txt")
qs = np.loadtxt("results/7quads_SNeIa_dg60_DESJ0408-5354_chi2.txt")

In [ ]:
%%script false
sorted_qidxs = np.argsort(qs)
print(sorted_qidxs[:10])
print([qs[i] for i in sorted_qidxs[:10]])


In [ ]:
# %%script false
reconsrc.chmdl(-1)
# reconsrc.chmdl(sorted_qidxs[0])
# reconsrc.chmdl(sorted_qidxs[1])
# reconsrc.chmdl(sorted_qidxs[3])

Mij = reconsrc.inv_proj_matrix(use_mask=True, r_max=2)
# Mij = reconsrc.inv_proj_matrix(use_mask=True)
# Mij = reconsrc.inv_proj_matrix(use_mask=False, r_max=10)
# Mij = reconsrc.inv_proj_matrix(use_mask=True, r_max=1.5)

In [ ]:
dij = reconsrc.lens_map(mask=True)
print("Calculating source plane...")
srcplane = reconsrc.d_p(method=kw['method'], flat=False, iter_lim=10000,
                        use_psf=kw['use_psf'], use_mask=kw['use_mask'], use_filter=kw['use_filter'],
                        sigma2=kw['sigma2'], cached=kw['cached'])
print("Calculating synthetic...")
synth = reconsrc.reproj_map(**kw)
# print("Calculating residuals...")
# resids = reconsrc.residual_map(**kw)
# print("Calculating chi2...")
# chi2 = reconsrc.reproj_chi2(reduced=False, **kw)
print(reconsrc.__v__)


In [ ]:
print("dij:      \t{}".format(dij.shape))
print("Mij:      \t{}".format(Mij.shape))
print("dp:       \t{}".format(srcplane.shape))
# print "chi2:     \t{:6.4f}".format(chi2)
print("r_max:    \t{:6.4f}".format(reconsrc.r_max))
print("r_fullres \t{:6.4f}".format(reconsrc.r_fullres))
print("src pxl:  \t{:6.4f}".format(reconsrc.src_pxscale))
print("maprad    \t{:6.4f}".format(reconsrc.lensobject.mapr))
print("lens pxl: \t{:6.4f}".format(reconsrc.lensobject.px2arcsec[0]))
sbf = (reconsrc.lensobject.px2arcsec[0]/reconsrc.src_pxscale)**2
print("SB scale factor: {:7.4f}".format(sbf))


In [ ]:
# Test flat dij with mask
flat_dij = reconsrc.d_ij(mask=True)
print(flat_dij.shape)
flat_dij = flat_dij.reshape(int(np.sqrt(flat_dij.shape[0])), int(np.sqrt(flat_dij.shape[0])))
print(flat_dij.shape)

In [ ]:
plt_kw = dict(origin='lower', cmap='gravic', interpolation='bicubic', vmin=0)
lvmax = 0.2500 * np.max(dij)      # 0.2500, 0.0075, 0.0075, 0.0750, 0.0750, 0.0100, 0.0100,  0.1000
svmax = sbf/1  * lvmax            # /4,     /1,     /1,     /16     /sbf    /2      **(-1)   /16

print("lvmax: {}".format(lvmax))
print("svmax: {}".format(svmax))

In [ ]:
plt.imshow(dij, extent=reconsrc.extent, vmax=lvmax, **plt_kw)
# # labels and annotations
plot_scalebar(R=reconsrc.lensobject.mapr, length=1)
plot_labelbox(lens, position='top left')

# # force mask
# m = reconsrc.image_mask()
# m = reconsrc.mask
# msk = np.zeros(m.shape + (4,))
# msk[:, :, 3] = m
# plt.imshow(msk, origin='lower', extent=lo.extent)

# mask cross
c = reconsrc.lensobject.roi.buffer.center
c = (c - reconsrc.lensobject.center.xy) * reconsrc.lensobject.px2arcsec[0]
plt.plot(*c, marker='+', markersize=10, color='grey')

plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.savefig("results/{}_data.pdf".format(lens), transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()


### Source plane

In [ ]:
plt.imshow(srcplane, extent=reconsrc.src_extent, vmax=1*svmax,
           **plt_kw)

# labels and annotations
plot_scalebar(R=reconsrc.r_max, length=1)
plot_labelbox(lens, position='top left')

plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.savefig("results/{}_{}_src.pdf".format(lens, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()


### Synthetic image

In [ ]:
plt.imshow(synth, extent=reconsrc.extent, vmax=0.75*lvmax, **plt_kw)

# labels and annotations
plot_scalebar(R=reconsrc.lensobject.mapr, length=1)
plot_labelbox(lens, position='top left')

# # force mask
# m = reconsrc.image_mask()
# m = reconsrc.mask
# msk = np.zeros(m.shape + (4,))
# msk[:, :, 3] = m
# plt.imshow(msk, origin='lower', extent=lo.extent)

# mask cross
c = reconsrc.lensobject.roi.buffer.center
c = (c - reconsrc.lensobject.center.xy) * reconsrc.lensobject.px2arcsec[0]
plt.plot(*c, marker='+', markersize=10, color='grey')

plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.savefig("results/{}_{}_best_synth.pdf".format(lens, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()


### Residual image

In [ ]:
kwp = plt_kw.copy()
kwp.pop('cmap')
kwp.pop('vmin')
# rmap = resids

f1, f2 = 1, 1
rmap = (f1*dij/dij.max() - f2*synth/synth.max())**2
rmap[rmap == 0] = rmap[rmap != 0].min()
rmap[rmap < 1e-4] = 1e-4
# print dij.min(), dij.max(), synth.min(), synth.max()
# print rmap


plt.imshow(rmap, extent=reconsrc.extent, cmap='gravic',
           norm=mpl.colors.LogNorm(vmin=1e-4, vmax=1e0), **kwp)

# labels and annotations
plot_scalebar(R=reconsrc.lensobject.mapr, length=1)
plot_labelbox(lens, position='top left')
plt.colorbar()

# force mask
# m = reconsrc.image_mask()
m = reconsrc.mask
msk = np.zeros(m.shape + (4,))
msk[:, :, 3] = m
plt.imshow(msk, origin='lower', extent=lo.extent)

# mask cross
c = reconsrc.lensobject.roi.buffer.center
c = (c - reconsrc.lensobject.center.xy) * reconsrc.lensobject.px2arcsec[0]
plt.plot(*c, marker='+', markersize=10, color='grey')

plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.savefig("results/{}_{}_resids.pdf".format(lens, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight')
plt.show()